In [ ]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, export_png
output_notebook()

from lsst.daf.persistence import Butler, NoResults
import os, numpy

In [ ]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/ts8/CALIB"
butler = Butler(DATADIR)

In [ ]:
camera = butler.get('camera')

In [ ]:
def make_subplot(plot, n):
    xmin = None
    xmax = None
    interp_list = []
    for k in qe_curve.data:
        wlen = qe_curve.data[k][0]
        eff = qe_curve.data[k][1]    
        plot.line(wlen.value, eff.value, alpha=0.1, line_width=3)
        
        amp_point = det[k].getBBox().getCenter()
        if xmin is None or xmax is None:
            xmin = wlen.min()
            xmax = wlen.max()
            interp_wlen = numpy.linspace(xmin, xmax, num=500)
        interp_eff = qe_curve.evaluate(det, amp_point, interp_wlen, kind='quadratic')
        interp_list.append(interp_eff)
        
    interp_arr = numpy.array([arr for arr in interp_list])
    median_eff = numpy.median(interp_arr, axis=0)
    plot.line(interp_wlen, median_eff, color='black', line_width=2)
    
    if n > 6:
        plot.xaxis.axis_label = 'wavelength (nm)'
        
    if n in (1, 4, 7):
        plot.yaxis.axis_label = 'efficiency (%)'
        
    plot.text(x=[400,], y=[20,], text=[f'Detector: {det.getName()}',])

In [ ]:
plots = []
for det in camera:
    rname, dname = det.getName().split('_')
    try:
        qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, taiObs='2000-01-01T00:00:00')
    except NoResults:
        continue  # No results found for this detector

    if len(plots) > 0:
        plots.append(figure(x_range=plots[0].x_range, y_range=plots[0].y_range))
    else:
        plots.append(figure())

    make_subplot(plots[-1], len(plots))

In [ ]:
grid = gridplot(plots, ncols=3, plot_width=500, plot_height=250)
show(grid)

## A rendered version of the above plot

![QE grid](qe_grid.png)